In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.MatchSRNN.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7281.82it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4522.13it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 3358771.46it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10566.69it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5337.62it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 838264.61it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 8968.00it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<0

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f1042783710>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f10503df400>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [7]:
padding_callback = mz.models.MatchSRNN.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.MatchSRNN()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['channels'] = 4
model.params['units'] = 10
model.params['dropout'] = 0.2
model.params['direction'] = 'lt'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

MatchSRNN(
  (embedding): Embedding(30058, 100)
  (dropout): Dropout(p=0.2, inplace=False)
  (matching_tensor_network): MatchingTensor()
  (spatial_gru_network): SpatialGRU(
    (_wr): Linear(in_features=34, out_features=30, bias=True)
    (_wz): Linear(in_features=34, out_features=40, bias=True)
    (_w_ij): Linear(in_features=4, out_features=10, bias=True)
    (_U): Linear(in_features=30, out_features=10, bias=False)
  )
  (out): Linear(in_features=10, out_features=1, bias=True)
)
Trainable params:  3048611


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-102 Loss-0.994]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.356 - normalized_discounted_cumulative_gain@5(0.0): 0.4454 - mean_average_precision(0.0): 0.414



[Iter-204 Loss-0.740]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5853 - normalized_discounted_cumulative_gain@5(0.0): 0.6469 - mean_average_precision(0.0): 0.6044



[Iter-306 Loss-0.544]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6018 - normalized_discounted_cumulative_gain@5(0.0): 0.6674 - mean_average_precision(0.0): 0.6209



[Iter-408 Loss-0.421]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5979 - normalized_discounted_cumulative_gain@5(0.0): 0.664 - mean_average_precision(0.0): 0.6192



[Iter-510 Loss-0.303]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5653 - normalized_discounted_cumulative_gain@5(0.0): 0.63 - mean_average_precision(0.0): 0.586



[Iter-612 Loss-0.229]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5929 - normalized_discounted_cumulative_gain@5(0.0): 0.6377 - mean_average_precision(0.0): 0.5971



[Iter-714 Loss-0.174]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5513 - normalized_discounted_cumulative_gain@5(0.0): 0.6166 - mean_average_precision(0.0): 0.5796



[Iter-816 Loss-0.117]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5744 - normalized_discounted_cumulative_gain@5(0.0): 0.6258 - mean_average_precision(0.0): 0.5884



[Iter-918 Loss-0.084]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5467 - normalized_discounted_cumulative_gain@5(0.0): 0.6106 - mean_average_precision(0.0): 0.5672



[Iter-1020 Loss-0.090]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5404 - normalized_discounted_cumulative_gain@5(0.0): 0.6036 - mean_average_precision(0.0): 0.558

Cost time: 4545.272604942322s
